In [1]:
%pprint

Pretty printing has been turned OFF


In [5]:
# load in spacy
import spacy
nlp = spacy.load('en')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
test = "I have a cat and that cat is my favorite cat in the whole wide world."
test2 = "Those cats I saw at a store down on Fifth Avenue were really the cutest cats I've ever seen in my life."
test3 = "I live on the Fifth Avenue."

# Spacy
one = []
t = nlp(test)
for tok in t:
    one.append((tok.text, tok.tag_))

two = []
t = nlp(test2)
for tok in t:
    two.append((tok.text, tok.tag_))

three = []
t = nlp(test3)
for tok in t:
    three.append((tok.text, tok.tag_))

In [6]:
one
two
three

[('I', 'PRP'), ('have', 'VBP'), ('a', 'DT'), ('cat', 'NN'), ('and', 'CC'), ('that', 'DT'), ('cat', 'NN'), ('is', 'VBZ'), ('my', 'PRP$'), ('favorite', 'JJ'), ('cat', 'NN'), ('in', 'IN'), ('the', 'DT'), ('whole', 'JJ'), ('wide', 'JJ'), ('world', 'NN'), ('.', '.')]

[('Those', 'DT'), ('cats', 'NNS'), ('I', 'PRP'), ('saw', 'VBD'), ('at', 'IN'), ('a', 'DT'), ('store', 'NN'), ('down', 'RP'), ('on', 'IN'), ('Fifth', 'NNP'), ('Avenue', 'NNP'), ('were', 'VBD'), ('really', 'RB'), ('the', 'DT'), ('cutest', 'JJS'), ('cats', 'NNS'), ('I', 'PRP'), ("'ve", 'VB'), ('ever', 'RB'), ('seen', 'VBN'), ('in', 'IN'), ('my', 'PRP$'), ('life', 'NN'), ('.', '.')]

[('I', 'PRP'), ('live', 'VBP'), ('on', 'IN'), ('the', 'DT'), ('Fifth', 'NNP'), ('Avenue', 'NNP'), ('.', '.')]

In [9]:
top = [1, 3, 7, 8, 3, -3, 3, 0]
for i in top:
    if i == 3:
        print("index:", top.index(i))
        print(i)
        print('----')

index: 1
3
----
index: 1
3
----
index: 1
3
----


In [7]:
hits = (i for i,value in enumerate(top) if value == 3)
for i in hits:
    print("index:", i)
    print(top[i])
    print('---')

index: 1
3
---
index: 4
3
---
index: 6
3
---


In [10]:
top = [(1, 'NN'), (3, 'd'), (7, 'NNS'), (8, 'NN'), (3, 'd'), (-3, 0), (3, 1), (0, 'l')]
hits = (i for i,value in enumerate(top) if value[1] == 'NN')
for i in hits:
    print("index:", i)
    print(top[i])
    print('---')

index: 0
(1, 'NN')
---
index: 3
(8, 'NN')
---


In [11]:
def findArticles(tokens):
    """Checks a list of tokens that have been POS-tagged with spaCyand are in the format [('tok', 'pos')...]. 
    Finds all nouns and looks at the word immediately before it to judge whether or not it is a determiner.
    Returns a list of all nouns and some basic information about them. The format of the returned list is:
    [('tok', isDefinite, isIndefinite, isZero, isPossessive, isPlural, isProper)], where everything after 'tok' is Boolean."""
    lst = []
    sing_reg = (i for i,value in enumerate(tokens) if value[1] == 'NN')
    pl_reg = (i for i,value in enumerate(tokens) if value[1] == 'NNS')
    sing_prop = (i for i,value in enumerate(tokens) if value[1] == 'NNP')
    pl_prop = (i for i,value in enumerate(tokens) if value[1] == 'NNPS')
    for w in tokens:
        # singular non-proper nouns
        for i in sing_reg:
            noun = tokens[i]
            prev = tokens[i-1]
            if prev[1] == 'DT':
                if prev[0] == 'The' or prev[0] == 'the':
                    lst.append((noun[0], 1, 0 , 0, 0, 0, 0))
                elif prev[0] == 'A' or prev[0] == 'An' or prev[0] == 'a' or prev[0] == 'an':
                    lst.append((noun[0], 0, 1 , 0, 0, 0, 0))
                else: lst.append((noun[0], 0, 0 , 0, 0, 0, 0))
            elif prev[1] == 'PRP$':
                lst.append((noun[0], 0, 0 , 0, 1, 0, 0))
            else:
                lst.append((noun[0], 0, 0 , 1, 0, 0, 0))
        # plural non-proper nouns
        for i in pl_reg:
            noun = tokens[i]
            prev = tokens[i-1]
            if prev[1] == 'DT':
                if prev[0] == 'The' or prev[0] == 'the':
                    lst.append((noun[0], 1, 0 , 0, 0, 1, 0))
                elif prev[0] == 'A' or prev[0] == 'An' or prev[0] == 'a' or prev[0] == 'an':
                    lst.append((noun[0], 0, 1 , 0, 0, 1, 0))
                else: lst.append((noun[0], 0, 0 , 0, 0, 1, 0))
            elif prev[1] == 'PRP$':
                lst.append((noun[0], 0, 0 , 0, 1, 1, 0))
            else:
                lst.append((noun[0], 0, 0 , 1, 0, 1, 0))        
        # singular proper nouns
        for i in sing_prop:
            noun = tokens[i]
            prev = tokens[i-1]
            if prev[1] != 'NNP':
                if prev[1] == 'DT':
                    if prev[0] == 'The' or prev[0] == 'the':
                        lst.append((noun[0], 1, 0 , 0, 0, 0, 1))
                    elif prev[0] == 'A' or prev[0] == 'An' or prev[0] == 'a' or prev[0] == 'an':
                        lst.append((noun[0], 0, 1 , 0, 0, 0, 1))
                    else: lst.append((noun[0], 0, 0 , 0, 0, 0, 1))
                elif prev[1] == 'PRP$':
                    lst.append((noun[0], 0, 0 , 0, 1, 0, 1))
                else:
                    lst.append((noun[0], 0, 0 , 1, 0, 0, 1))                                    
            else: 
                two = tokens[i-2]
                if two[1] == 'DT':
                    if two[0] == 'The' or two[0] == 'the':
                        lst.append((noun[0], 1, 0 , 0, 0, 0, 1, 'long'))
                    elif two[0] == 'A' or two[0] == 'An' or two[0] == 'a' or two[0] == 'an':
                        lst.append((noun[0], 0, 1 , 0, 0, 0, 1, 'long'))
                    else: lst.append((noun[0], 0, 0 , 0, 0, 0, 1, 'long'))
                elif two[1] == 'PRP$':
                    lst.append((noun[0], 0, 0 , 0, 1, 0, 1, 'long'))
                else:
                    lst.append((noun[0], 0, 0 , 1, 0, 0, 1, 'long'))
        # plural proper nouns
        for i in pl_prop:
            noun = tokens[i]
            prev = tokens[i-1]
            if prev[1] != 'NNP':
                if prev[1] == 'DT':
                    if prev[0] == 'The' or prev[0] == 'the':
                        lst.append((noun[0], 1, 0 , 0, 0, 1, 1))
                    elif prev[0] == 'A' or prev[0] == 'An' or prev[0] == 'a' or prev[0] == 'an':
                        lst.append((noun[0], 0, 1 , 0, 0, 1, 1))
                    else: lst.append((noun[0], 0, 0 , 0, 0, 1, 1))
                elif prev[1] == 'PRP$':
                    lst.append((noun[0], 0, 0 , 0, 1, 1, 1))
                else:
                    lst.append((noun[0], 0, 0 , 1, 0, 1, 1))
            else: 
                two = tokens[i-2]
                if two[1] == 'DT':
                    if two[0] == 'The' or two[0] == 'the':
                        lst.append((noun[0], 1, 0 , 0, 0, 1, 1, 'long'))
                    elif two[0] == 'A' or two[0] == 'An' or two[0] == 'a' or two[0] == 'an':
                        lst.append((noun[0], 0, 1 , 0, 0, 1, 1, 'long'))
                    else: lst.append((noun[0], 0, 0 , 0, 0, 1, 1, 'long'))
                elif two[1] == 'PRP$':
                    lst.append((noun[0], 0, 0 , 0, 1, 1, 1, 'long'))
                else:
                    lst.append((noun[0], 0, 0 , 1, 0, 1, 1, 'long'))
    long = (i for i,value in enumerate(lst) if value[-1] == 'long')
    for i in long:
        prev = lst[i-1]
        lst.remove(prev)
    return lst

In [12]:
one
findArticles(one)

[('I', 'PRP'), ('have', 'VBP'), ('a', 'DT'), ('cat', 'NN'), ('and', 'CC'), ('that', 'DT'), ('cat', 'NN'), ('is', 'VBZ'), ('my', 'PRP$'), ('favorite', 'JJ'), ('cat', 'NN'), ('in', 'IN'), ('the', 'DT'), ('whole', 'JJ'), ('wide', 'JJ'), ('world', 'NN'), ('.', '.')]

[('cat', 0, 1, 0, 0, 0, 0), ('cat', 0, 0, 0, 0, 0, 0), ('cat', 0, 0, 1, 0, 0, 0), ('world', 0, 0, 1, 0, 0, 0)]

In [13]:
two
findArticles(two)

[('Those', 'DT'), ('cats', 'NNS'), ('I', 'PRP'), ('saw', 'VBD'), ('at', 'IN'), ('a', 'DT'), ('store', 'NN'), ('down', 'RP'), ('on', 'IN'), ('Fifth', 'NNP'), ('Avenue', 'NNP'), ('were', 'VBD'), ('really', 'RB'), ('the', 'DT'), ('cutest', 'JJS'), ('cats', 'NNS'), ('I', 'PRP'), ("'ve", 'VB'), ('ever', 'RB'), ('seen', 'VBN'), ('in', 'IN'), ('my', 'PRP$'), ('life', 'NN'), ('.', '.')]

[('store', 0, 1, 0, 0, 0, 0), ('life', 0, 0, 0, 1, 0, 0), ('cats', 0, 0, 0, 0, 1, 0), ('cats', 0, 0, 1, 0, 1, 0), ('Avenue', 0, 0, 1, 0, 0, 1, 'long')]

In [14]:
three
findArticles(three)

[('I', 'PRP'), ('live', 'VBP'), ('on', 'IN'), ('the', 'DT'), ('Fifth', 'NNP'), ('Avenue', 'NNP'), ('.', '.')]

[('Avenue', 1, 0, 0, 0, 0, 1, 'long')]